# 21) Introduction to MPI.jl

Last time:
- Slurm demo
- Parallel linear algebra

Today:
1. Introduction to MPI.jl
2. Setup MPI.jl
3. Hello world
4. Setup on tuckoo cluster

## 1. Introduction to MPI.jl

Recap on MPI so far:

- Each process executes its own program with its own data (**MPMD: Multiple Program, Multiple Data**)
- Each process can communicate with every other process through messages (chunks of binary data, generally arrays of primitive data types like `int`, `double`, etc..)
- Messages can be _point-to-point_ (between only two processes) or _collective_ such as _broadcasts_ (one to all) and _reductions_ (all to one).
- Though in principle each process can execute a unique program, generally each process executes the same program but the work it does depends on it's unique number within all the processes.

**Terminology**:

- **communicator**: a group of processes that can communicate (a process can be in many different communicators, though in this class we will generally just use a single global communicator).

- **rank**: the unique identifying number of a process within a communicator. Process ranks run from 0 to the number of processes minus one in the communicator. In general, a processes rank will have a different in each communicator it is a member of.

[MPI.jl](https://juliaparallel.org/MPI.jl/stable/) is a basic Julia wrapper for the MPI standard. 

## 2. Setup MPI.jl

To be able to run MPI instructions in Julia, we first need to [configure MPI.jl](https://juliaparallel.org/MPI.jl/stable/configuration/).

To do so, we need the auxiliary package MPIPreferences.jl. This will (hopefully) detect an MPI implementation library on your system at one of the default locations. 

To do so, launch Julia and in the REPL do the following:

```bash
julia
```

```julia
julia> using Pkg; Pkg.add("MPIPreferences.jl")

julia> using MPIPreferences; MPIPreferences.use_system_binary()
```

The call to `use_system_binary()` should automatically detect the MPI binary. 

Now you should be ready to run your first MPI.jl hello world program.



## 3. Hello world

In the Julia REPL, you can do:

```julia
julia> using Pkg; Pkg.add("MPI.jl")

julia> using MPI; mpiexec(cmd -> run(`$cmd -n 3 echo hello world`))
```

Or you can find the following example in [`julia_codes/module6-1/`](https://github.com/sdsu-comp605/spring25/tree/main/julia_codes/module6-1/).


```{literalinclude} ../julia_codes/module6-1/mpi_hello_world.jl
:language: julia
:linenos: true
```

Once you have setup your environment via MPIPreferences and added MPI, you can run the above example under your `mpiexec`:


```bash
mpiexec -n 4 julia ./julia_codes/module6-1/mpi_hello_world.jl
```

## 4. Setup on tuckoo cluster

On the cluster, you can simply start julia by typing `julia` from any location. 

We will still use MPIPreferences to set up our environment, but this time the call to `use_system_binary()` instead of having an empty default list of arguments, will include the argument `library_names` with the location of where the MPI binary resides on the parallel file system in the cluster.

```
$ julia

julia> using Pkg; Pkg.add("MPIPreferences")
julia> using MPIPreferences
julia> MPIPreferences.use_system_binary(library_names="/usr/lib64/openmpi/lib/libmpi.so")
julia> using Pkg; Pkg.add("MPI")
```

Once your environment is setup on the cluster, you can run with 

```bash
mpiexec -np 4 julia mpi_hello_world.jl
```

You can find an example of a batch script that launches a julia job in `/examples/slurm/batch-jello`

```bash
#!/bin/sh

#note: run julia hello-world on a single node

#SBATCH --job-name=jhello
#SBATCH --output=%A-jhello.out
#SBATCH --ntasks=16

export OMPI_MCA_pml=ob1
export OMPI_MCA_btl=tcp,self

#require tcp over infiniband
export OMPI_MCA_btl_tcp_if_include ib0

mpirun julia mpi_hello_world.jl
```